In [ ]:
!pip install fiftyone
!pip install ultralytics
!pip install gradio

### Download Potholes-dataset

Reference : https://public.roboflow.com/object-detection/pothole/1/download/yolov8

In [ ]:
import os
import shutil

dataset_name = "potholes-dataset"
os.makedirs(dataset_name, exist_ok=True) 

!gdown --id 1loWMnhgYpCGLwHKNwMZS1FlWmfkFSEiq
shutil.move("Pothole.v1-raw.yolov8.zip", dataset_name)

# For windows
!tar -xf {dataset_name}/Pothole.v1-raw.yolov8.zip -C "{dataset_name}"

# For linux
# !unzip {dataset_name}/Pothole.v1-raw.yolov8.zip -d "{dataset_name}"

## Training Detection Model

Yolov8 - utralytics : https://docs.ultralytics.com/tasks/detect/

> editing `data.yaml` file

example > train: `../train/images` --> `/home/badboy-005/Desktop/community-notebooks/potholes-detection/potholes-dataset/test`

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml')  # build a new model from YAML
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# Train the model for 50 epochs
results = model.train(data="./potholes-dataset/data.yaml", epochs=50, imgsz=640)

## Inference using gradio

In [ ]:
import gradio as gr
from PIL import Image, ImageDraw, ImageFont

In [ ]:
def inference(gr_input):
    """
    Inference function for gradio.
    """
    pred = model(gr_input)
    draw_prediction = ImageDraw.Draw(gr_input)
    boxes_predict = pred[0].boxes
    boxes = boxes_predict.xyxy.tolist()
    scores = boxes_predict.conf.tolist()
    for score, box in zip(scores, boxes):
        x, y, x2, y2 = tuple(box)
        draw_prediction.rectangle((x, y, x2, y2), outline="red", width=2)
    return gr_input

In [ ]:
model = YOLO("runs/detect/train3/weights/best.pt")
imagein = gr.inputs.Image(label="Input Image", type="pil")
imageout = gr.outputs.Image(label="Predicted Image", type="pil")

interface = gr.Interface(
    fn=inference,
    inputs=imagein,
    outputs=imageout,
    title="Potholes detection",
    interpretation="default",
).launch(debug="True")

# launch demo
interface.launch()